# Getting to know the environment


In [1]:
import gymnasium as gym
import pandas as pd
import numpy as np

In [5]:
# Build an environment with gym.make()
env = gym.make('FrozenLake-v1',render_mode = "human") # Build a fresh environment

# # Start a new game with env.reset()
current_observation = env.reset() # This starts a new "episode" and returns the initial observation

# # The current observation is just the current location
print(current_observation) # Observations are just a number

(0, {'prob': 1})


In [6]:
# There are fource discrete actions available in this environment
print(f'Our action space: {env.action_space}')

# Randomly sample an action and print the result
new_action = env.action_space.sample()
print(f'Our new action: {new_action}')

Our action space: Discrete(4)
Our new action: 0


In the following code, we will let the agent traverse through an entire episode in a random manner. 

An episode will end when a termination condition is met. A termination condition happens when the agent either falls into a hole or reaches the goal

In [14]:
num_episodes = 10

for episode in range(num_episodes):
    # Reset the testing environment and termination condition for every training episode
    current_observation = env.reset()
    done = False

    while not(done):
        # Sample a random action and take the action to observe the result
        new_action = env.action_space.sample()
        observation, reward, term, trunc, info = env.step(new_action)

        # Print out the result of the action
        print(f"Episode: {episode+1}, Action: {new_action}, observation: {observation}, reward: {reward}, terminated: {term}, truncated: {trunc}, info: {info}")

        # Check the termination condition
        done = term or trunc


Episode: 1, Action: 0, observation: 0, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode: 1, Action: 0, observation: 0, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode: 1, Action: 0, observation: 0, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode: 1, Action: 3, observation: 0, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode: 1, Action: 3, observation: 0, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode: 1, Action: 2, observation: 0, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode: 1, Action: 1, observation: 0, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode: 1, Action: 2, observation: 4, reward: 0.0, terminated: False, truncated: False, info: {'prob': 0.3333333333333333}
Episode:

# Step 1 - Gather Data
In the last section, we have explored how to simulate episodes of the game from the start to a termination point. In this section, we would like to gather more of these episodes to start training an intelligent actor.

We will let the system simulate 1000 episodes of the game and collect the data using pandas. Each row of the data frame is a step, while the features of each rows are

- **`observation`** - the observation at the beginning of the step (before taking the action)
- **`action`** - the randomly sampled action
- **`DCF_reward`** - the distributed reward after each step leading to a final reward is taken. In other words, if the episode leads to the reward, current_reward is the final reward distributed to each step.
- **`reward`** - the accumulated reward per episode. Subsequent to simulating every episode, this is used to calculate the success rate of reaching the goal

Also consider a reward **`success`** that has a value of 1 at each step if the whole episode leads to final reward

In [15]:
import numpy as np
import pandas as pd
import gymnasium as gym

In [39]:
env = gym.make("FrozenLake-v1",render_mode = "rgb_array")

# Intialize a list for storing the trainng data
train_data = []  

num_episodes = 40000
for episode in range(num_episodes):
    initial_observation = env.reset()[0]
    done = False
    cumsum_reward = 0 # Accumulated reward for each episode
    step = 0
    ep_data = []

    while not(done):
        # Sample a random action in the action space and take a step
        new_action = env.action_space.sample()
        new_observation, current_reward, term, trunc, info = env.step(new_action)

        cumsum_reward += current_reward
        step += 1
        done = term or trunc

        # Append a dictionary of key-value pairs to the list
        ep_data.append({
            # 'eps': episode,
            'step': step,
            'prevObs': initial_observation,
            'act': new_action,
            'currObs': new_observation,
            'reward': current_reward,
            'done': done
        })

        initial_observation = new_observation

    # Adding a linearly discounted reward
    num_steps = len(ep_data)
    for idx, step in enumerate(ep_data):
        step['success'] = cumsum_reward                         # To specify the steps that led to a reward
        step["DCF_reward"] = idx /num_steps * cumsum_reward     # The reward distributed evenly through the steps

    train_data += ep_data

train_data_df = pd.DataFrame(train_data)
pd.set_option('display.max_rows', None)

# df.describe()

reward_per_eps = train_data_df['reward'].sum()/(episode+1)*100
print(f"\n Success chance: {reward_per_eps}%")


 Success chance: 1.3925%


# Step 2 - Predict

In this section, we are hoping to leverage the training data collected in the last example to improve the performance of our agent. We will adopt a supervised learning approach, where the input of the model is a combination of observation and action, and the model output is the expected reward of each of the possible action given a certain observation. 

In [38]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR

In [ ]:
model = ExtraTreesRegressor(n_estimators = 50)
# model = SVR()

# The proposed structure of reward, weighting the three rewards accordingly
    # The first component of the reward happens at goal states
    # The second component of the rewards encourages actions that leads closer to the reward (since these states have higher DCF_reward)
    # The third component of the reward encourages any action in a sequence in a successful episode
y = 0.5 * train_data_df.reward + 0.1 * train_data_df.DCF_reward + train_data_df.success


x = train_data_df[["prevObs", "act"]]

model.fit(x, y)

# Step 3 - Act
Now we have a model that predicts the desired behaviour, trained to estimate a reward value when given an observation and an action. We can modify the code used in the data gathering step so that we can replace random actions with more educated ones.

With a random agent, we achieved a success rate of 1-2%. With more informed actions, the success rate should increase by 10x. 

How we can improve the model:
- We can further tune the model for better performance. 
- Try different representations of the observation and actions spaces. 
- Try different models 
- Try different reward structures

In [43]:
model = RandomForestRegressor()
# model = ExtraTreesRegressor(n_estimators = 50)

y = 0.5 * train_data_df.reward + 0.1 * train_data_df.DCF_reward + train_data_df.success
x = train_data_df[['prevObs','act']]
model.fit(x,y)

RandomForestRegressor()

In [49]:
num_episodes = 100

test_data = []  

# Create the testing environment. Set render_mode to rgb_array for faster testing, set render_mode to human for visualization (slower performance)
env_test = gym.make("FrozenLake-v1",render_mode = "human")
# env_test = gym.make("FrozenLake-v1",render_mode = "rgb_array")

for episode in range(num_episodes):
    
    initial_observation = env_test.reset()[0]
    done = False
    cumsum_reward = 0 # Accumulated reward for each episode
    step = 0
    ep_data = []

    # print(f"Episode: {episode+1}")

    while not(done):
        # Sample a random action in the action space and take a step
        new_action = env_test.action_space.sample()

        # # Make informed choice from the trained model
        pred_in = pd.DataFrame([[initial_observation, i] for i in range(4)], columns = ['prevObs','act']) # This is the ['prevObs','act'] pair that is fed into the model
        new_action = np.argmax(model.predict(pred_in))

        # Take the step and record the result of the step
        new_observation, current_reward, term, trunc, info = env_test.step(new_action)

        cumsum_reward += current_reward
        step += 1
        done = term or trunc

        # Append a dictionary of key-value pairs to the list
        ep_data.append({
            # 'eps': episode,
            'step': step,
            'prevObs': initial_observation,
            'act': new_action,
            'currObs': new_observation,
            'reward': current_reward,
            'done': done
        })

        initial_observation = new_observation

    # Adding a linearly discounted reward
    num_steps = len(ep_data)
    for idx, step in enumerate(ep_data):
        step['success'] = cumsum_reward                         # To specify the steps that led to a reward
        step["DCF_reward"] = idx /num_steps * cumsum_reward     # The reward distributed evenly through the steps

    test_data += ep_data


KeyboardInterrupt: 

In [48]:
test_data_df = pd.DataFrame(test_data)

reward_per_eps = test_data_df['reward'].sum()/(num_episodes+1)*100
print(f"\n Success chance: {reward_per_eps}%")


 Success chance: 61.386138613861384%
